In [1]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup


mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [3]:
alphavantage_csv_path = "/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AMZN.csv"

In [4]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date"] = pd.to_datetime(df['date'])

df.set_index("date", inplace=True)

In [5]:
df.shape[0]

3064

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   1. open               3064 non-null   float64
 1   2. high               3064 non-null   float64
 2   3. low                3064 non-null   float64
 3   4. close              3064 non-null   float64
 4   5. adjusted close     3064 non-null   float64
 5   6. volume             3064 non-null   float64
 6   7. dividend amount    3064 non-null   float64
 7   8. split coefficient  3064 non-null   float64
dtypes: float64(8)
memory usage: 215.4 KB


In [7]:
df.head(1)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2010-01-04,136.25,136.61,133.14,133.9,133.9,7599900.0,0.0,1.0


In [8]:
benzinga_csv_path = "/app/StockPricePredictions/data/benzinga/amzn_non_dupes.csv"

In [9]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [10]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12394 entries, 0 to 12393
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      12394 non-null  object
 1   author    12394 non-null  object
 2   created   12394 non-null  object
 3   updated   12394 non-null  object
 4   title     12394 non-null  object
 5   teaser    3464 non-null   object
 6   body      10367 non-null  object
 7   url       12394 non-null  object
 8   image     12394 non-null  object
 9   channels  12394 non-null  object
 10  stocks    12394 non-null  object
 11  tags      12394 non-null  object
 12  AAPL      12394 non-null  bool  
 13  MSFT      12394 non-null  bool  
 14  AMZN      12394 non-null  bool  
 15  FB        12394 non-null  bool  
 16  NVDA      12394 non-null  bool  
 17  INTC      12394 non-null  bool  
 18  NFLX      12394 non-null  bool  
dtypes: bool(7), object(12)
memory usage: 1.2+ MB


In [11]:
df_benzinga.head(2)

,date,author,created,updated,title,teaser,body,url,image,channels,stocks,tags,AAPL,MSFT,AMZN,FB,NVDA,INTC,NFLX
0,2010-01-28,Benzinga,"Thu, 28 Jan 2010 15:29:57 -0400","Thu, 28 Jan 2010 15:29:57 -0400","Benzinga’s Volume Movers (FFBC, NFLX, GNTX, ALGN, UTHR)",NaN,"First Financial Bancorp (NASDAQ: <a class=""ticker"" href=""/stock/ffbc#NASDAQ"">FFBC</a>) shares gained 6.94% to $16.19. The volume of FFBC shares traded was 1311% higher than normal. FFBC has entered into an underwriting agreement for the sale of about 85 million common shares.\r\n\r\nNetflix Inc (NASDAQ: <a class=""ticker"" href=""/stock/nflx#NASDAQ"">NFLX</a>) shares jumped 22.48% to $62.43. The volume of NFLX shares traded was 804% higher than normal. NFLX has hinted at expanding its operations to overseas markets.\r\n\r\nGentex Corporation (NASDAQ: <a class=""ticker"" href=""/stock/gntx#NASDAQ"">GNTX</a>) shares climbed 8.02% to $18.85. The volume of GNTX shares traded was 372% higher than normal. Robert W Baird has upgraded GNTX from “neutral” to “outperform.”\r\n\r\nAlign Technology Inc (NASDAQ: <a class=""ticker"" href=""/stock/algn#NASDAQ"">ALGN</a>) shares rose 11.44% to $18.60. The volume of ALGN shares traded was 355% higher than normal. GNTX has posted a 45% increase in its net sales for Q4.\r\n\r\nUnited Therapeutics Corporation (NASDAQ: <a class=""ticker"" href=""/stock/uthr#NASDAQ"">UTHR</a>) shares moved up 2.90% to $59.23. The volume of UTHR shares traded was 350% higher than normal. UTHR’s shares have surpassed their ten-day moving average of $58.23.\r\n\r\nRead more <a href=""http://www.benzinga.com/markets"">from Benzinga's Markets.</a>",https://www.benzinga.com/node/103495,[],"[{'name': 'Long Ideas'}, {'name': 'News'}, {'name': 'Intraday Update'}, {'name': 'Markets'}, {'name': 'Movers'}, {'name': 'Trading Ideas'}]","[{'name': 'ALGN'}, {'name': 'FFBC'}, {'name': 'GNTX'}, {'name': 'NFLX'}, {'name': 'UTHR'}]","[{'name': 'Auto Parts & Equipment'}, {'name': 'Biotechnology'}, {'name': 'Consumer Discretionary'}, {'name': 'Financials'}, {'name': 'Health Care'}, {'name': 'Health Care Supplies'}, {'name': 'Internet Retail'}, {'name': 'Regional Banks'}, {'name': 'volume movers'}]",False,False,False,True,False,False,True
1,2010-03-10,Benzinga,"Wed, 10 Mar 2010 15:03:42 -0400","Wed, 10 Mar 2010 15:24:17 -0400","Benzinga's Top ETF Gainers (FBT, FAS, USD, KBE)",NaN,"Biotech stocks are having a good trading session today. InterMune, Inc. (NASDAQ: <a class=""ticker"" href=""/stock/itmn#NASDAQ"">ITMN</a>) spiked 63% after the company won approval of an advisory panel to launch a new drug. InterMune (<a class=""ticker"" href=""/stock/itmn#"">ITMN</a>) is one of the holdings of First Trust NYSE Arca Biotechnology(<a class=""ticker"" href=""/stock/etf#"">ETF</a>) (NYSE: <a class=""ticker"" href=""/stock/fbt#NYSE"">FBT</a>). FBT is trading 3.31% up, and its last transaction was made at $37.16.\r\n\r\nDirexion Daily Financial Bull 3X Shares(<a class=""ticker"" href=""/stock/etf#"">ETF</a>) (NYSE: <a class=""ticker"" href=""/stock/fas#NYSE"">FAS</a>) is also trading in the positive territory. FAS is 3% up, along with Bank of America Corporation (NYSE: <a class=""ticker"" href=""/stock/bac#NYSE"">BAC</a>) and Wells Fargo & Company (NYSE: <a class=""ticker"" href=""/stock/wfc#NYSE"">WFC</a>). Bank of america (<a class=""ticker"" href=""/stock/bac#"">BAC</a>) trades with a gain of 2.32% and Wells Fargo (<a class=""ticker"" href=""/stock/wfc#"">WFC</a>) added 2.17% on the upside.\r\n\r\nIntel Corporation (NASDAQ: <a class=""ticker"" href=""/stock/intc#NASDAQ"">INTC</a>) and Texas Instruments Incorporated (NYSE: <a class=""ticker"" href=""/stock/txn#NYSE"">TXN</a>) are trading in the positive territory, just as ProShares Ultra Semiconductors (<a class=""ticker"" href=""/stock/etf#"">ETF</a>) (NYSE: <a class=""ticker"" href=""/stock/usd#NYSE"">USD</a>). USD added 2.87% so far, while Intel (<a class=""ticker"" href=""/stock/intc#"">INTC</a>) and Texas Instruments (<a class=""

In [12]:
df_benzinga["body"] = df_benzinga["body"].replace(np.nan, "")

In [13]:
def clean_text(text):
    soup = BeautifulSoup(text)

    # remove HTML tables
    for table in soup.find_all("table"):
        table.extract()

    text_without_tables = soup.text

    pattern1 = re.compile(r"\s")

    clean_text1 = re.sub(pattern1, " ", text_without_tables)

    pattern2 = re.compile(r"\\r|\\+n")

    clean_text2 = re.sub(pattern1, " ", clean_text1)

    clean_text3 = re.sub(r'^https?:\/\/.*[\r\n]*', '', clean_text2, flags=re.MULTILINE)

    return clean_text3

    
    

In [14]:
df_benzinga["body"] = df_benzinga["body"].apply(clean_text)

/root/.local/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.ifixit.com/Teardown/Oculus+Rift+Teardown/13682" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/root/.local/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "http://www.ftc.gov/news-events/press-releases/2014/04/ftc-notifies-facebook-whatsapp-privacy-obligations-light-proposed" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/root/.local/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://www.moves-app.com/press" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client li

In [15]:
df_benzinga["body"] = df_benzinga["body"].replace(np.nan, "")

In [16]:
df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [17]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['date'])

In [18]:
df_benzinga.set_index("date", inplace=True)

In [19]:
df_benzinga = df_benzinga[["text"]]

In [20]:
def remove_ticker(text):

    clean_text1 = re.sub(r'\(NASDAQ\s?:\s?\w+\)', '', text, flags=re.MULTILINE)

    clean_text2 = re.sub(r'\(NYSE\s?:\s?\w+\)', '', clean_text1, flags=re.MULTILINE)

    return clean_text2


In [21]:
df_benzinga["text"] = df_benzinga["text"].apply(remove_ticker)

In [22]:
benzinga_csv_out_path = "/app/StockPricePredictions/data/benzinga/amzn_non_dupes_text_only.csv"

In [24]:
# df_benzinga.to_csv(benzinga_csv_out_path)
df_benzinga2 = pd.read_csv(benzinga_csv_out_path)


In [25]:
df_benzinga2.head()

date  \
0  2010-01-28   
1  2010-03-10   
2  2010-03-29   
3  2010-04-01   
4  2010-05-17   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [643]:
# import spacy

In [644]:
# nlp = spacy.load('en_core_web_md')
# myvals = df_benzinga['text'].values.tolist()

In [645]:
# df_benzinga['len'] = df_benzinga['text'].str.split().str.len()

In [646]:
# df_benzinga.describe()